# Evaluación y Comparativa de Modelos de Lenguaje a Gran Escala (LLM) en Local para Arquitecturas RAG: Efectividad, Privacidad y Autonomía

Este notebook constituye el espacio de trabajo donde se han desarrollado y ejecutado los experimentos presentados en este estudio. Aquí se integra todo el marco experimental, desde la configuración inicial del entorno hasta la ejecución y análisis de los flujos RAG, utilizando las interfaces diseñadas específicamente para este propósito. 

En cada sección se abordan las distintas etapas del experimento, incluyendo la carga de datos, la construcción de pipelines RAG y la evaluación mediante las métricas definidas en el marco RAGAs. Este notebook no solo documenta los resultados obtenidos, sino que también sirve como referencia técnica para replicar y ajustar los experimentos en futuros estudios.

## Configuración del Entorno

Antes de ejecutar este notebook, asegúrate de que tu entorno está correctamente configurado para evitar problemas de compatibilidad o interferencias con otros proyectos. A continuación, te indicamos los pasos necesarios para el set up:

1. **Instalación de Python 3.12**:  
   Este notebook requiere Python 3.12. Descárgalo e instálalo desde [python.org](https://www.python.org/).

2. **Crear un entorno virtual (recomendado)**:
    Para evitar conflictos con otras configuraciones de Python, se recomienda crear un entorno virtual. Puedes hacerlo con los siguientes comandos:

    ```
    py -3.12 -m venv venv
    source venv/bin/activate  # En Linux/Mac
    venv\Scripts\activate     # En Windows 
    ```

3. **Instalación de iKernel**:  
   Para utilizar Python 3.12 como motor del notebook, necesitas instalar el módulo `ipykernel`. Esto puede hacerse directamente desde el entorno virtual mediante el siguiente comando:  

   ```
   pip install ipykernel
   ```

   Asegúrate de añadir este kernel al notebook una vez instalado.

4. **Instalación de dependencias**:
    Las dependencias necesarias para este notebook se encuentran en la celda de configuración. Estas se instalarán directamente desde el notebook mediante `%pip install...` Si las instalas por primera vez, es posible que necesites reiniciar el kernel para que surtan efecto.

**Nota importante:** Durante la instalación de las librerías necesarias en este entorno virtual, es posible que aparezcan mensajes relacionados con incompatibilidades entre algunas dependencias. Por ejemplo, conflictos menores entre las versiones de `tenacity`, `grpcio`, `protobuf` y `deepeval`. Estas incompatibilidades son inherentes a las dependencias actuales de las librerías utilizadas en este notebook, pero hemos verificado que no afectan su ejecución ni los resultados esperados.

In [ ]:
## ----------------------------------------------------------------------------
## CONFIGURACIÓN DEL ENTORNO: 
# Instalación de paquetes y librerías necesarias
## ----------------------------------------------------------------------------

# Update pip
%pip install -qU pip

# Langchain Installation
%pip install -qU langchain langchain_community

# PDF Loaders Libraries
%pip install -qU pymupdf 
%pip install -qU langchain-unstructured "langchain-unstructured[local]" unstructured-client unstructured "unstructured[pdf]" 
%pip install -qU azure-ai-documentintelligence azure-core
%pip install -qU langchain-text-splitters

# Embeddings Models Libraries
%pip install -qU langchain_ollama
%pip install -qU langchain-openai
%pip install -qU langchain_voyageai
%pip install -qU langchain_mistralai

# Vector Store Libraries
%pip install -qU langchain_chroma
%pip install -qU langchain-mongodb pymongo

# Large Language Models Libraries
%pip install -qU langchain-ollama
%pip install -qU langchain-openai
%pip install -qU langchain_anthropic
%pip install -qU langchain-google-genai
%pip install -qU langchain_mistralai

# Evaluation Libraries
%pip install -qU deepeval
%pip install -qU instructor

# Other Libraries
%pip install -qU python-dotenv
%pip install -qU ipywidgets
%pip install -qU tqdm
%pip install -qU transformers

## Configuración de las Variables de Entorno

Este notebook requiere varias claves de API y configuraciones específicas. Por favor, copia el archivo `.env_sample`, renómbralo a `.env` y edítalo con tus claves y configuraciones.

In [1]:
## ----------------------------------------------------------------------------
## CONFIGURACIÓN DE LAS VARIABLES DE ENTORNO: 
# Carga de variables de entorno desde el archivo .env
## ----------------------------------------------------------------------------

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", verbose=True)

True

## Interfaz para los Módulos de Vectorización

En este apartado, construiremos nuestra interfaz de embeddings personalizada en base a los conectores de embeddings proporcionados por LangChain. Gracias a que LangChain actúa como capa de abstracción, todos los conectores funcionan de manera homogénea, lo que nos permite intercambiarlos sin complicaciones. De esta forma, podremos cargar dinámicamente distintos módulos de embeddings según lo definan nuestras variables, facilitando la integración y flexibilidad en nuestros experimentos.

In [2]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Vectorización
# Nomic Embeddings: nomic-embed-text - LOCAL [768]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model nomic-embed-text from https://ollama.com/library/nomic-embed-text:latest
3. Start the Ollama server
'''

from langchain_ollama import OllamaEmbeddings

def myNomicEmbedder():
    embedder = OllamaEmbeddings(model="nomic-embed-text")
    return embedder

## # Test Embedder
## input_text = "The meaning of life is 42"
## embedder = myNomicEmbedder()
## vector = embedder.embed_query(input_text)
## print(vector[:3])

In [3]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Vectorización
# Snowflake Embeddings: snowflake-arctic-embed:335m - LOCAL [1024]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model snowflake-arctic-embed from https://ollama.com/library/snowflake-arctic-embed
3. Start the Ollama server
'''

from langchain_ollama import OllamaEmbeddings

def mySnowflakeArcticEmbedder():
    embedder = OllamaEmbeddings(model="snowflake-arctic-embed:335m")
    return embedder

## # Test Embedder
## input_text = "The meaning of life is 42"
## embedder = mySnowflakeArcticEmbedder()
## vector = embedder.embed_query(input_text)
## print(vector[:3])

In [4]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Vectorización
# OpenAI Embeddings: text-embedding-3-large - CLOUD [3072]
## ----------------------------------------------------------------------------

'''
1. Create an account in OpenAI API Platform: https://platform.openai.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_openai import OpenAIEmbeddings

def myOpenAiEmbedder():
    embedder = OpenAIEmbeddings(model="text-embedding-3-large")
    return embedder

## # Test Embedder
## input_text = "The meaning of life is 42"
## embedder = myOpenAiEmbedder()
## vector = embedder.embed_query(input_text)
## print(vector[:3])

In [5]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Vectorización
# Voyage AI Embeddings: voyage-3 - CLOUD [1024]
## ----------------------------------------------------------------------------

'''
1. Create an account in Voyage AI Platform: https://www.voyageai.com
2. Create an API Key
3. Set up the API Key in the .env file
'''

import warnings
from tqdm import TqdmWarning
warnings.filterwarnings("ignore", category=TqdmWarning)

import os
from langchain_voyageai import VoyageAIEmbeddings

def myVoyageAiEmbedder():
    embedder = VoyageAIEmbeddings(voyage_api_key=os.getenv("VOYAGEAI_API_KEY"), model="voyage-3")
    return embedder

## # Test Embedder
## input_text = "The meaning of life is 42"
## embedder = myVoyageAiEmbedder()
## vector = embedder.embed_query(input_text)
## print(vector[:3])

In [6]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Vectorización
# Mistral AI Embeddings: mistral-embed - CLOUD [1024]
## ----------------------------------------------------------------------------

'''
1. Create an account in Mistral AI Platform: https://console.mistral.ai
2. Create an API Key
3. Set up the API Key in the .env file
4. You might need as well a Hugging Face Token. Set it up in the .env file
'''

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="langchain_mistralai.embeddings")

from langchain_mistralai import MistralAIEmbeddings

def myMistralAiEmbedder():
    embedder = MistralAIEmbeddings(model="mistral-embed")
    return embedder

## # Test Embedder
## input_text = "The meaning of life is 42"
## embedder = myMistralAiEmbedder()
## vector = embedder.embed_query(input_text)
## print(vector[:3])

In [7]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE VECTORIZACIÓN:
# Interfaz personalizada para definir de manera dinámica el modulo de 
# vectorización.

# Modelos Disponibles:
# - nomic-embed-text
# - snowflake-arctic-embed-335m
# - text-embedding-3-large
# - voyage-3
# - mistral-embed
## ----------------------------------------------------------------------------

class MyEmbedder:

    def __init__(self, model: str):
        self.model = model
        self.dimension = {
            "nomic-embed-text": 768,
            "snowflake-arctic-embed-335m": 1024,
            "text-embedding-3-large": 3072,
            "voyage-3": 1024,
            "mistral-embed": 1024
        }.get(model, None)
        
    def get_embedder(self):
        if self.model == "nomic-embed-text":
            return myNomicEmbedder()
        elif self.model == "snowflake-arctic-embed-335m":
            return mySnowflakeArcticEmbedder()
        elif self.model == "text-embedding-3-large":
            return myOpenAiEmbedder()
        elif self.model == "voyage-3":
            return myVoyageAiEmbedder()
        elif self.model == "mistral-embed":
            return myMistralAiEmbedder()
        else:
            raise ValueError(f"Model {self.model} not supported")

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE VECTORIZACIÓN:
# Prueba de la interfaz personalizada para los módulos de vectorización.
## ----------------------------------------------------------------------------

# Define the embedding model to use
# Options: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3', 'mistral-embed'
embedding_model = 'text-embedding-3-large'
embedder = MyEmbedder(model=embedding_model).get_embedder()

# Test the embedder
input_text = 'The meaning of life is 42'
embedding = embedder.embed_query(input_text)
print(f'Lenght of the vector: {len(embedding)}.\nFirst 3 elements: {embedding[:3]} ...')

## Interfaz para los Módulos de Bases de Datos de Vectores

En esta sección, definiremos nuestras bases de datos de vectores y los retrievers asociados utilizando los conectores disponibles en LangChain. Aunque todos los conectores de bases de datos de vectores comparten una interfaz común, presentan diferencias en cómo filtran o eliminan la información, lo que nos obliga a implementar algunas variaciones en estos procesos. A pesar de estas diferencias, LangChain sigue actuando como una capa de abstracción que facilita la integración y el intercambio de bases de datos de vectores, permitiéndonos cargarlas dinámicamente según lo definan nuestras variables y garantizando la flexibilidad en nuestros experimentos.

In [8]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Bases de Datos de Vectores
# Chroma Vector Store - LOCAL
## ----------------------------------------------------------------------------

'''
1. The Chroma Vector Store is a local database for storing and querying vectors.
2. The objects are stored by default in the directory "../ChromaDb".
'''

from langchain_chroma import Chroma

def myChromaDbVectorStore(Embedder: MyEmbedder):
    vector_store = Chroma(
        collection_name=f'collection-{Embedder.model}',
        embedding_function=Embedder.get_embedder(),
        persist_directory="../ChromaDb",
        # Available functions: 'l2', 'cosine', and 'ip'
        collection_metadata={"hnsw:space": "cosine"} 
    )
    return vector_store

In [9]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Bases de Datos de Vectores
# MongoDB Atlas Vector Store - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in MongoDB Atlas: https://www.mongodb.com/cloud/atlas
2. Create a Cluster
3. Update the .env file with the MongoDB User, Password, and Cluster URI
4. First time setting up a collection, set create_index=True
'''

import os
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

def myMongoDbAtlasVectorStore(Embedder: MyEmbedder, create_index: bool = False):
    client = MongoClient(os.getenv("MONGO_DB_CLUSTER_URI"))

    DB_NAME = "myMongoDbAtlasVectorStore"
    COLLECTION_NAME = f"collection-{Embedder.model}"
    ATLAS_VECTOR_SEARCH_INDEX_NAME = f"index-{Embedder.model}"

    MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

    vector_store = MongoDBAtlasVectorSearch(
        collection=MONGODB_COLLECTION,
        embedding=Embedder.get_embedder(),
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
        # Available functions: 'euclidean', 'cosine', and 'dotProduct'
        relevance_score_fn="cosine", 
    )

    # Create the vector search index - Needs to be done on the initial setup
    if create_index:
        vector_store.create_vector_search_index(dimensions=Embedder.dimension, filters=["source", "parser"])

    return vector_store

In [10]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE BASES DE DATOS DE VECTORES:
# Interfaz personalizada para definir de manera dinámica el modulo de base 
# de datos de vectores.

# Proveedores Disponibles:
# - chromadb
# - mongodb
## ----------------------------------------------------------------------------

from uuid import uuid4

class MyVectorStore:

    def __init__(self, provider: str, Embedder: MyEmbedder):
        self.provider = provider
        self.Embedder = Embedder
    
    def get_vector_store(self):
        if self.provider == "chromadb":
            return myChromaDbVectorStore(Embedder=self.Embedder)
        elif self.provider == "mongodb":
            return myMongoDbAtlasVectorStore(Embedder=self.Embedder)
        else:
            raise ValueError(f"Provider {self.provider} not supported")

    def get_vector_store_documents(self, source: str, parser: str):
        vector_store = self.get_vector_store()

        if self.provider == "chromadb":
            old_docs = vector_store.get(where={"$and": [{"source": source}, {"parser": parser}]})
            return old_docs["ids"]
        
        elif self.provider == "mongodb":
            old_docs = list(vector_store._collection.find({"source": source, "parser": parser}, {"_id": 1}))
            return [doc["_id"] for doc in old_docs]

    def purge_vector_store_documents(self, source: str, parser: str):
        vector_store = self.get_vector_store()
        document_ids = self.get_vector_store_documents(source=source, parser=parser)
        if document_ids:
            vector_store.delete(ids=document_ids)
    
    def push_vector_store_documents(self, documents: list, embeddings: list = None):
        vector_store = self.get_vector_store()
        if not embeddings:
            vector_store.add_documents(documents=documents)
            return
        
        if self.provider == "chromadb":
            vector_store._collection.add(
                ids=[str(uuid4()) for _ in range(len(documents))],
                documents=[doc.page_content for doc in documents],
                embeddings=embeddings,
                metadatas=[doc.metadata for doc in documents]
            )
        
        elif self.provider == "mongodb":
            vector_store._collection.insert_many([
                {
                    "text" : text,
                    "embedding" : embedding,
                    **metadata
                } for text, embedding, metadata in zip(
                    [doc.page_content for doc in documents], 
                    embeddings, 
                    [doc.metadata for doc in documents]
                )
            ])

In [11]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE RETRIEVAL:
# Interfaz personalizada para definir de manera dinámica el modulo de 
# retrieval.
## ----------------------------------------------------------------------------

class MyRetriever:

    def __init__(self, k: int, source: str, parser: str, VectorStore: MyVectorStore):
        self.k = k
        self.source = source
        self.parser = parser
        self.VectorStore = VectorStore

    def get_retriever(self):
        vector_store = self.VectorStore.get_vector_store()
        if self.VectorStore.provider == "chromadb":
            return vector_store.as_retriever(search_type='similarity', search_kwargs={'k': self.k, 'filter': {"$and": [{"source": self.source}, {"parser": self.parser}]}})
        elif self.VectorStore.provider == "mongodb":
            return vector_store.as_retriever(search_type='similarity', search_kwargs={'k': self.k, 'pre_filter': {"source": {"$eq": self.source}, "parser": {"$eq": self.parser}}})
        else:
            raise ValueError(f"Provider {self.provider} not supported")

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE BASES DE DATOS DE VECTORES:
# Prueba de la interfaz personalizada para los módulos de base de 
# datos de vectores - Carga de documentos.
## ----------------------------------------------------------------------------

from uuid import uuid4
from langchain_core.documents import Document

# Define the documents to add to the vector store
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "test", "parser": "test"}
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "test", "parser": "test"}
)

document_3 = Document(
    page_content="There has been a significant increase in the number of COVID-19 cases in the past week.",
    metadata={"source": "test", "parser": "test"}
)

documents = [document_1, document_2, document_3]
uuids = [str(uuid4()) for _ in range(len(documents))]

# Load the documents into the vector stores
# Available providers: 'chromadb', 'mongodb'
for db_provider in ["chromadb", "mongodb"]:
    # Using all the possible embedding models
    # Available models: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3', 'mistral-embed'
    for embedding_model in ["nomic-embed-text", "snowflake-arctic-embed-335m", "text-embedding-3-large", "voyage-3", "mistral-embed"]:
        
        # Define the Embedder and Vector Store
        Embedder = MyEmbedder(model=embedding_model)
        VectorStore = MyVectorStore(provider=db_provider, Embedder=Embedder)

        # Purge Existing Documents
        VectorStore.purge_vector_store_documents(source="test", parser="test")
        
        # Embed the Documents
        embedder = Embedder.get_embedder()
        embeddings = [embedder.embed_query(doc.page_content) for doc in documents]
        
        # Push the Documents
        VectorStore.push_vector_store_documents(documents=documents, embeddings=embeddings)

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE RETRIEVAL:
# Prueba de la interfaz personalizada para los módulos de retrieval - Busqueda 
# de documentos.
## ----------------------------------------------------------------------------

# Define the Embedder
# Available models: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3', 'mistral-embed'
embedding_model = "text-embedding-3-large"
Embedder = MyEmbedder(model=embedding_model)

# Define the Vector Store
# Available providers: 'chromadb', 'mongodb'
db_provider = "mongodb"
VectorStore = MyVectorStore(provider=db_provider, Embedder=Embedder)

# Define the Retriever and its filters
k, source, parser = 2, "test", "test"
retriever = MyRetriever(k=k, source=source, parser=parser, VectorStore=VectorStore).get_retriever()

# Test the retriever
query = "Will it be hot tomorrow?"
docs = retriever.invoke(query)
print(docs[0].page_content)

## Interfaz para los Módulos de LLMs

En esta sección, desarrollaremos nuestra interfaz personalizada para los LLMs utilizando los conectores disponibles a través de LangChain. Al igual que con los embeddings, LangChain actúa como una capa de abstracción, lo que garantiza que todos los conectores de LLMs funcionen de manera homogénea. Esto nos permite intercambiar entre distintos modelos de lenguaje sin dificultades. Gracias a esta flexibilidad, podemos cargar dinámicamente diferentes modelos de LLM según lo definan nuestras variables, facilitando la integración y adaptación de nuestros experimentos.

In [12]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# OpenAI: gpt-4o - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in OpenAI API Platform: https://platform.openai.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_openai import ChatOpenAI

def myOpenAiLLM_gpt_4o(temperature: float):
    return ChatOpenAI(model="gpt-4o", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myOpenAiLLM_gpt_4o(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [13]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# OpenAI: o1-preview - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in OpenAI API Platform: https://platform.openai.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_openai import ChatOpenAI

def myOpenAiLLM_o1(temperature: float):
    return ChatOpenAI(model="o1-preview", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myOpenAiLLM_o1(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [14]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Anthropic: claude-3-5-haiku - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Anthropic AI Platform: https://www.anthropic.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_anthropic import ChatAnthropic

def myAnthropicLLM_claude_3_5_haiku(temperature: float):
    return ChatAnthropic(model="claude-3-5-haiku-latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myAnthropicLLM_claude_3_5_haiku(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [15]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Anthropic: claude-3-5-sonnet - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Anthropic AI Platform: https://www.anthropic.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_anthropic import ChatAnthropic

def myAnthropicLLM_claude_3_5_sonnet(temperature: float):
    return ChatAnthropic(model="claude-3-5-sonnet-latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myAnthropicLLM_claude_3_5_sonnet(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [16]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Anthropic: claude-3-opus - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Anthropic AI Platform: https://www.anthropic.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_anthropic import ChatAnthropic

def myAnthropicLLM_claude_3_opus(temperature: float):
    return ChatAnthropic(model="claude-3-opus-latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myAnthropicLLM_claude_3_opus(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [17]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Google: gemini-1.5-flash - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Google API Platform: https://aistudio.google.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_google_genai import ChatGoogleGenerativeAI

def myGoogleLLM_gemini_1_5_flash(temperature: float):
    return ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myGoogleLLM_gemini_1_5_flash(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [18]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Google: gemini-1.5-pro - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Google API Platform: https://aistudio.google.com/
2. Create an API Key
3. Set up the API Key in the .env file
'''

from langchain_google_genai import ChatGoogleGenerativeAI

def myGoogleLLM_gemini_1_5_pro(temperature: float):
    return ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myGoogleLLM_gemini_1_5_pro(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [19]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Google: gemma2:2b - LOCAL [1.6Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model gemma2:2b from https://ollama.com/library/gemma2
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myGoogleLLM_gemma2_2b(temperature: float):
    return ChatOllama(model="gemma2:2b", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myGoogleLLM_gemma2_2b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [20]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Google: gemma2:9b - LOCAL [5.4Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model gemma2:9b from https://ollama.com/library/gemma2
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myGoogleLLM_gemma2_9b(temperature: float):
    return ChatOllama(model="gemma2:latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myGoogleLLM_gemma2_9b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [21]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Google: gemma2:27b - LOCAL [16Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model gemma2:27b from https://ollama.com/library/gemma2
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myGoogleLLM_gemma2_27b(temperature: float):
    return ChatOllama(model="gemma2:27b", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myGoogleLLM_gemma2_27b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [22]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Meta: llama3.2:1b - LOCAL [1.3Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model llama3.2:1b from https://ollama.com/library/llama3.2
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myMetaLLM_llama3_2_1b(temperature: float):
    return ChatOllama(model="llama3.2:1b", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMetaLLM_llama3_2_1b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [23]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Meta: llama3.2:3b - LOCAL [2Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model llama3.2:3b from https://ollama.com/library/llama3.2
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myMetaLLM_llama3_2_3b(temperature: float):
    return ChatOllama(model="llama3.2:3b", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMetaLLM_llama3_2_3b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [24]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Meta: llama3.1:8b - LOCAL [4.7Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model llama3.1:8b from https://ollama.com/library/llama3.1
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myMetaLLM_llama3_1_8b(temperature: float):
    return ChatOllama(model="llama3.1:8b", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMetaLLM_llama3_1_8b(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [25]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Nvidia: nemotron-mini - LOCAL [2.7Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model nemotron-mini from https://ollama.com/library/nemotron-mini
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myNvidiaLLM_nemotron_mini(temperature: float):
    return ChatOllama(model="nemotron-mini:latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myNvidiaLLM_nemotron_mini(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [26]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Microsoft: phi3.5 - LOCAL [2.2Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model phi3.5 from https://ollama.com/library/phi3.5
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myMicrosoftLLM_phi3_5(temperature: float):
    return ChatOllama(model="phi3.5:latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMicrosoftLLM_phi3_5(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [27]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Mistral AI: mistral-large - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Mistral AI Platform: https://console.mistral.ai
2. Create an API Key
3. Set up the API Key in the .env file
4. You might need as well a Hugging Face Token. Set it up in the .env file
'''

from langchain_mistralai import ChatMistralAI

def myMistralLLM_mistral_large(temperature: float):
    return ChatMistralAI(model="mistral-large-latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMistralLLM_mistral_large(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [28]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de LLM
# Mistral AI: mistral-small - LOCAL [13Gb]
## ----------------------------------------------------------------------------

'''
1. Download Ollama from https://ollama.com
2. Download the model mistral-small from https://ollama.com/library/mistral-small
3. Start the Ollama server
'''

from langchain_ollama import ChatOllama

def myMistralLLM_mistral_small(temperature: float):
    return ChatOllama(model="mistral-small:latest", temperature=temperature)

## # Test LLM
## messages = [("human", "What is the meaning of life?")]
## llm = myMistralLLM_mistral_small(temperature=1)
## response = llm.invoke(messages)
## print(response.content)

In [29]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE LLM:
# Interfaz personalizada para definir de manera dinámica el modulo de LLM.

# Modelos Disponibles:
# - gpt-4o, o1-preview
# - claude-3-5-haiku, claude-3-5-sonnet, claude-3-opus
# - gemini-1.5-flash, gemini-1.5-pro
# - gemma2-2b, gemma2-9b, gemma2-27b
# - llama3-2-1b, llama3-2-3b, llama3-1-8b
# - nemotron-mini
# - phi3-5
# - mistral-large, mistral-small
## ----------------------------------------------------------------------------

class MyLLM:

    def __init__(self, model: str, temperature: float):
        self.model = model
        self.temperature = temperature
    
    def get_llm(self):
        if self.model == "gpt-4o":
            return myOpenAiLLM_gpt_4o(temperature=self.temperature)
        elif self.model == "o1-preview":
            return myOpenAiLLM_o1(temperature=self.temperature)
        elif self.model == "claude-3-5-haiku":
            return myAnthropicLLM_claude_3_5_haiku(temperature=self.temperature)
        elif self.model == "claude-3-5-sonnet":
            return myAnthropicLLM_claude_3_5_sonnet(temperature=self.temperature)
        elif self.model == "claude-3-opus":
            return myAnthropicLLM_claude_3_opus(temperature=self.temperature)
        elif self.model == "gemini-1.5-flash":
            return myGoogleLLM_gemini_1_5_flash(temperature=self.temperature)
        elif self.model == "gemini-1.5-pro":
            return myGoogleLLM_gemini_1_5_pro(temperature=self.temperature)
        elif self.model == "gemma2-2b":
            return myGoogleLLM_gemma2_2b(temperature=self.temperature)
        elif self.model == "gemma2-9b":
            return myGoogleLLM_gemma2_9b(temperature=self.temperature)
        # Not working in my laptop due to RAM limitations
        ## elif self.model == "gemma2-27b":
        ##     return myGoogleLLM_gemma2_27b(temperature=self.temperature)
        elif self.model == "llama3-2-1b":
            return myMetaLLM_llama3_2_1b(temperature=self.temperature)
        elif self.model == "llama3-2-3b":
            return myMetaLLM_llama3_2_3b(temperature=self.temperature)
        elif self.model == "llama3-1-8b":
            return myMetaLLM_llama3_1_8b(temperature=self.temperature)
        elif self.model == "nemotron-mini":
            return myNvidiaLLM_nemotron_mini(temperature=self.temperature)
        elif self.model == "phi3-5":
            return myMicrosoftLLM_phi3_5(temperature=self.temperature)
        elif self.model == "mistral-large":
            return myMistralLLM_mistral_large(temperature=self.temperature)
        # Ollama distribution for this model is not working
        ## elif self.model == "mistral-small":
        ##     return myMistralLLM_mistral_small(temperature=self.temperature)
        else:
            raise ValueError(f"Model {self.model} not supported")

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE LLM:
# Prueba de la interfaz personalizada para los módulos de LLM.
## ----------------------------------------------------------------------------

# Define the LLM model to use
# Options: 'gpt-4o', 'o1-preview', 'claude-3-5-haiku', 'claude-3-5-sonnet', 'claude-3-opus', 'gemini-1.5-flash', 'gemini-1.5-pro', 
# 'gemma2-2b', 'gemma2-9b', 'llama3-2-1b', 'llama3-2-3b', 'llama3-1-8b', 'nemotron-mini', 'phi3-5', 'mistral-large'
llm_model, temperature = "gpt-4o", 1
llm = MyLLM(model=llm_model, temperature=temperature).get_llm()

# Test the LLM
messages = [("human", "What is the meaning of life?")]
response = llm.invoke(messages)
print(response.content)

## Interfaz para el Procesamiento de PDFs

En esta sección, construiremos nuestra interfaz personalizada para el procesamiento de PDFs. Su función principal será extraer el contenido de un documento PDF y fragmentarlo en unidades de información, que luego serán cargadas en las bases de datos de vectores. Aunque LangChain proporciona algunos conectores para el procesamiento de PDFs, no todos los conectores están disponibles ni existe una interfaz estándar. Por lo tanto, seremos responsables de crear esta interfaz, homogenizando tanto la entrada como la salida, para asegurar que todos los módulos de procesamiento de PDFs funcionen de manera consistente en nuestros experimentos.

In [30]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# PyMuPDF Loader - LOCAL
## ----------------------------------------------------------------------------

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def myPyMuPDFLoader(path, chunk_size=500, chunk_overlap=50):
    
    loader = PyMuPDFLoader(file_path=path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(loader.load())

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myPyMuPDFLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [31]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# Unstructured IO Loader - LOCAL
## ----------------------------------------------------------------------------

'''
1. Create an account in Unstructured IO Platform: https://unstructured.io/
2. Create an API Key
3. Set up the API Key in the .env file
4. Install Docker from https://www.docker.com/get-started
5. Pull the Unstructured IO Docker image: docker pull downloads.unstructured.io/unstructured-io/unstructured-api:latest
6. Run the Unstructured IO Docker image: docker run -p 9500:9500 -d --rm --name unstructured-api -e PORT=9500 downloads.unstructured.io/unstructured-io/unstructured-api:latest
'''

from langchain_unstructured import UnstructuredLoader
from unstructured_client import UnstructuredClient

def myUnstructuredLocalLoader(path, chunk_size=500, chunk_overlap=50):

    client = UnstructuredClient(
        server_url="http://localhost:9500"
    )
    
    loader = UnstructuredLoader(
        file_path=path, 
        partition_via_api=True, 
        client=client, 
        strategy="hi_res", 
        chunking_strategy="by_title", 
        max_characters=chunk_size,
        overlap=chunk_overlap,
        include_orig_elements=False
    )
    
    return loader.load()

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myUnstructuredLocalLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [32]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# Unstructured IO Loader - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Unstructured IO Platform: https://unstructured.io/
2. Create an API Key
3. Set up the API Key in the .env file
'''

import os
from langchain_unstructured import UnstructuredLoader
from unstructured_client import UnstructuredClient

def myUnstructuredCloudLoader(path, chunk_size=500, chunk_overlap=50):

    client = UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY"), 
        server_url="https://api.unstructuredapp.io/general/v0/general"
    )
    
    loader = UnstructuredLoader(
        file_path=path, 
        partition_via_api=True, 
        client=client, 
        strategy="hi_res", 
        chunking_strategy="by_title", 
        max_characters=chunk_size,
        overlap=chunk_overlap,
        include_orig_elements=False
    )
    
    return loader.load()

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myUnstructuredCloudLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [33]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# Azure Document Intelligence Loader - LOCAL
## ----------------------------------------------------------------------------

'''
1. Create an account in Azure: https://portal.azure.com/#home
2. Create a Document Intelligence Resource
3. Set up the API Key and Endpoint URI in the .env file
4. Install Docker from https://www.docker.com/get-started
5. Update your API Key in the ../AzDocInt/compose.yml file
6. Pull and run the Azure Document Intelligence Docker image from ../AzDocInt: docker compose up
'''

import os
import json
import requests

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def myAzureDocIntLocalLoader(path, chunk_size=500, chunk_overlap=50):

    # Extract text from the PDF
    with open(path, "rb") as f:           
        result = requests.post(
            url = f"{os.getenv('DOCUMENT_INTELLIGENCE_LOCAL_ENDPOINT_URI')}/formrecognizer/documentModels/prebuilt-layout:syncAnalyze?api-version=2023-07-31",
            headers={"Content-Type": "application/octet-stream"},
            files={"file": f}
        )
        
    result = json.loads(result.text)['analyzeResult']

    # Split the text into documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = [Document(page_content=content) for content in text_splitter.split_text(result['content'])]
    return documents

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myAzureDocIntLocalLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [34]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# Azure Document Intelligence Loader - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Azure: https://portal.azure.com/#home
2. Create a Document Intelligence Resource
3. Set up the API Key and Endpoint URI in the .env file
'''

import os

from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat

def myAzureDocIntCloudLoader(path, chunk_size=500, chunk_overlap=50):
    
    # Extract text from the PDF
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=os.getenv('DOCUMENT_INTELLIGENCE_CLOUD_ENDPOINT_URI'), 
        credential=AzureKeyCredential(os.getenv('DOCUMENT_INTELLIGENCE_API_KEY'))
    )

    with open(path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", output_content_format=ContentFormat.MARKDOWN
        )
    
    result: AnalyzeResult = poller.result()

    # Split the text into documents
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers = False)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    return text_splitter.split_documents(markdown_splitter.split_text(result.content)) 

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myAzureDocIntCloudLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [35]:
## ----------------------------------------------------------------------------
## Interfaz para los Módulos de Procesamiento de PDFs
# LlamaParse Loader - CLOUD
## ----------------------------------------------------------------------------

'''
1. Create an account in Llama AI Platform: https://llamaindex.ai/
2. Create an API Key
3. Set up the API Key in the .env file
'''

import os
import time
import requests

from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

def myLlamaParseLoader(path, chunk_size=500, chunk_overlap=50):

    # Upload the file
    with open(path, 'rb') as f:
        response = requests.post(
            url = 'https://api.cloud.llamaindex.ai/api/parsing/upload',
            data={
                'continuous_mode': True, 
                'premium_mode': True
            },
            headers={
                'accept': 'application/json',
                'Authorization': f'Bearer {os.getenv("LLAMA_CLOUD_API_KEY")}'
            },
            files={'file': (os.path.basename(path), f, 'application/pdf')}
        )

    response.raise_for_status()
    job_id = response.json()['id']

    # Check the status of the parsing job
    while True:
        response = requests.get(
            url=f'https://api.cloud.llamaindex.ai/api/parsing/job/{job_id}',
            headers={
                'accept': 'application/json',
                'Authorization': f'Bearer {os.getenv("LLAMA_CLOUD_API_KEY")}'
            }
        )

        response.raise_for_status()

        status = response.json()['status']
        if status == 'SUCCESS':
            break
        time.sleep(5)

    # Get the results in Markdown
    response = requests.get(
        url=f'https://api.cloud.llamaindex.ai/api/parsing/job/{job_id}/result/markdown',
        headers={
            'accept': 'application/json',
            'Authorization': f'Bearer {os.getenv("LLAMA_CLOUD_API_KEY")}'
        }
    )
    
    response.raise_for_status()
    result = response.json()['markdown']

    # Split the text into documents
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers = False)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    return text_splitter.split_documents(markdown_splitter.split_text(result))

## # Test Loader
## path = r"../PDFs/Attention Is All You Need.pdf"
## documents = myLlamaParseLoader(path)
## print(f"Loaded {len(documents)} documents from {path}")

In [36]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE PROCESAMIENTO DE PDFs:
# Interfaz personalizada para definir de manera dinámica el modulo de 
# procesamiento de PDFs. 

# Métodos Disponibles:
# - extract: Extrae el texto de un PDF utilizando el servicio seleccionado.
# - load: Carga los documentos extraidos en el Vector Store seleccionado.

# Servicios Disponibles:
# - PyMuPDF
# - UnstructuredLocal, UnstructuredCloud
# - AzureDocIntLocal, AzureDocIntCloud
# - LlamaParse
## ----------------------------------------------------------------------------

import os

class MyPdfLoader:
        
        def __init__(self, path: str, loader: str, chunk_size=500, chunk_overlap=50):
            self.path = path
            self.loader = loader
            self.chunk_size = chunk_size
            self.chunk_overlap = chunk_overlap
            self.source = os.path.basename(self.path)
            
        def extract(self):
    
            # Pdf Extraction
            if self.loader == "PyMuPDF":
                docs = myPyMuPDFLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            elif self.loader == "UnstructuredLocal":
                docs = myUnstructuredLocalLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            elif self.loader == "UnstructuredCloud":
                docs = myUnstructuredCloudLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            elif self.loader == "AzureDocIntLocal":
                docs = myAzureDocIntLocalLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            elif self.loader == "AzureDocIntCloud":
                docs = myAzureDocIntCloudLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            elif self.loader == "LlamaParse":
                docs = myLlamaParseLoader(self.path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            else:
                raise ValueError(f"Unknown loader: {self.loader}")

            return docs
        
        def load(self, VectorStore: MyVectorStore, documents: list = None, embeddings: list = None, replace: bool = False):

            # Purge Existing Documents if replace=True
            document_ids = VectorStore.get_vector_store_documents(source=self.source, parser=self.loader)
            if document_ids:
                if replace:
                    VectorStore.purge_vector_store_documents(source=self.source, parser=self.loader)
                else:
                    print(f"Documents already exist. Set replace=True to overwrite.")
                    return

            # Extract and Load Documents
            if not documents:
                documents = self.extract()
            
            for document in documents: 
                document.metadata = {"source": self.source, "parser": self.loader}
            
            VectorStore.push_vector_store_documents(documents=documents, embeddings=embeddings)
            

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA LOS MÓDULOS DE PROCESAMIENTO DE PDFs:
# Prueba de la interfaz personalizada para los módulos de procesamiento de PDFs.
## ----------------------------------------------------------------------------

# Define the PDF to load from the PDFs folder
# Available PDFs: 'Attention Is All You Need.pdf', 'CODIMUR 50.pdf', 'Orden ayudas publicas.pdf', 'PPT Sostenibilidad.pdf', 'UOC Ciencia de Datos.pdf'
path = r"../PDFs/Attention Is All You Need.pdf"

# Define the Embedder
# Available models: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3', 'mistral-embed'
embedding_model = "text-embedding-3-large"
Embedder = MyEmbedder(embedding_model)

# Define the Vector Store
# Available providers: 'chromadb', 'mongodb'
db_provider = "mongodb"
VectorStore = MyVectorStore(provider=db_provider, Embedder=Embedder)

# Define the PDF Loader
# Available loaders: 'PyMuPDF', 'UnstructuredLocal', 'UnstructuredCloud', 'AzureDocIntLocal', 'AzureDocIntCloud', 'LlamaParse'
pdf_loader = "PyMuPDF"
PdfLoader = MyPdfLoader(path=path, loader=pdf_loader)

# Extract and Load the PDF data into the vector store
PdfLoader.load(VectorStore=VectorStore, replace=True)

## Flujo de Carga de Documentos PDF

En esta sección, definimos el flujo de carga que se encarga de procesar iterativamente todos los PDFs, considerando todas las combinaciones posibles de Loader, Embedder y Vector Store. Este flujo es crucial para llenar nuestras bases de datos de vectores, que servirán como fuente de conocimiento para nuestro conjunto de RAGs.

In [ ]:
## ----------------------------------------------------------------------------
## FLUJO DE CARGA DE DOCUMENTOS PDF:
# Procesamiento y carga de los documentos PDF de manera iterativa para todas 
# las combinaciones de los módulos de nuestro estudio.
## ----------------------------------------------------------------------------

import os

import logging
logging.getLogger('requests').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# For all the pdfs
# Available PDFs: 'Attention Is All You Need.pdf', 'CODIMUR 50.pdf', 'Orden ayudas publicas.pdf', 'PPT Sostenibilidad.pdf', 'UOC Ciencia de Datos.pdf'
pdfs_list = [r"../PDFs/Attention Is All You Need.pdf", r"../PDFs/CODIMUR 50.pdf", r"../PDFs/Orden ayudas publicas.pdf", r"../PDFs/PPT Sostenibilidad.pdf", r"../PDFs/UOC Ciencia de Datos.pdf"]
for path in pdfs_list:
       
    # For all the pdf loaders
    # Available loaders: 'PyMuPDF', 'AzureDocIntLocal', 'LlamaParse', 'UnstructuredCloud'
    pdf_loader_list = ["PyMuPDF", "AzureDocIntLocal", "LlamaParse", "UnstructuredCloud"]
    for pdf_loader in pdf_loader_list:

        print(f"Extracting {os.path.basename(path)} with {pdf_loader} ...")
        PdfLoader = MyPdfLoader(path=path, loader=pdf_loader)
        documents = PdfLoader.extract()
        print(f"Extracting {os.path.basename(path)} with {pdf_loader} ... DONE")

        # For all the embedding models
        # Available models: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3'
        embedding_models_list = ["nomic-embed-text", "snowflake-arctic-embed-335m", "text-embedding-3-large", "voyage-3"]
        for embedding_model in embedding_models_list:

            print(f"Embedding {os.path.basename(path)} with {embedding_model} ...")
            Embedder = MyEmbedder(embedding_model)
            embeddings = [Embedder.get_embedder().embed_query(doc.page_content) for doc in documents]
            print(f"Embedding {os.path.basename(path)} with {embedding_model} ... DONE")

            # For all the database providers
            # Available providers: 'chromadb', 'mongodb'
            db_providers_list = ["chromadb", "mongodb"]
            for db_provider in db_providers_list:

                print(f"Loading {os.path.basename(path)} into {db_provider} ...")
                VectorStore = MyVectorStore(provider=db_provider, Embedder=Embedder)
                PdfLoader.load(VectorStore=VectorStore, documents=documents, embeddings=embeddings, replace=False)
                print(f"Loading {os.path.basename(path)} into {db_provider} ... DONE")
                
                print(f"Processing {os.path.basename(path)} with {pdf_loader} and {embedding_model} into {db_provider} ... DONE")

## Interfaz para el Flujo RAG

En esta sección, definimos la interfaz del flujo RAG, que permite construir y ejecutar fácilmente un conjunto de RAGs. Este flujo toma como entrada los módulos configurados previamente, como el procesador de PDFs, el modelo de embeddings, la base de datos de vectores y el modelo de lenguaje. Además, permite definir parámetros como el número de resultados a recuperar (top k) y la temperatura del modelo de lenguaje para personalizar las respuestas generadas.

In [37]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA EL FLUJO RAG:
# Interfaz personalizada para definir de manera dinámica el flujo de RAG a 
# partir de sus componentes.
## ----------------------------------------------------------------------------

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


class MyRAGChain:

    def __init__(self, source: str, pdf_loader: str, embedding_model: str, db_provider: str, llm_model: str, k: int = 5, temperature: float = 0.5):
        self.source = source
        self.pdf_loader = pdf_loader
        self.embedding_model = embedding_model
        self.db_provider = db_provider
        self.llm_model = llm_model
        self.k = k
        self.temperature = temperature
    
    def get_rag_chain(self):
        
        # Document Formatting
        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        ## Prompt Template for RAG
        RAG_TEMPLATE = """
        You are an assistant for question-answering tasks. 
        Use only the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 

        <context>
        {context}
        </context>

        Answer the following question:

        {question}"""

        rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

        # Retriever and Top K
        Embedder = MyEmbedder(model=self.embedding_model)
        VectorStore = MyVectorStore(provider=self.db_provider, Embedder=Embedder)
        retriever = MyRetriever(k=self.k, source=self.source, parser=self.pdf_loader, VectorStore=VectorStore).get_retriever()

        # Language Model and Temperature
        llm = MyLLM(model=self.llm_model, temperature=self.temperature).get_llm()

        # Langchain Pipeline
        qa_chain = (
            {
                "retrievals": retriever,
                "question": RunnablePassthrough(),
            }
            | RunnableMap(
                {
                    "retrievals": lambda inputs: inputs["retrievals"],
                    "answer": (
                        {
                            "context": lambda inputs: format_docs(inputs["retrievals"]),
                            "question": lambda inputs: inputs["question"],
                        }
                        | rag_prompt
                        | llm
                        | StrOutputParser()
                    ),
                }
            )
            | (lambda outputs: {"retrievals": outputs["retrievals"], "answer": outputs["answer"]})
        )

        return qa_chain

In [ ]:
## ----------------------------------------------------------------------------
## INTERFAZ PARA EL FLUJO RAG:
# Prueba de la interfaz personalizada para el flujo de RAG.
## ----------------------------------------------------------------------------

# Define the embedding model to use
# Options: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3'
embedding_model = "text-embedding-3-large"

# Define the database provider to use
# Options: 'chromadb', 'mongodb'
db_provider = "mongodb"

# Define the LLM model to use
# Options: 'gpt-4o', 'claude-3-5-sonnet', 'gemini-1.5-pro', gemma2-9b, llama3-1-8b, phi3-5
llm_model, temperature = "gpt-4o", 0.5

# Define the PDF Loader to use
# Options: 'PyMuPDF', 'UnstructuredCloud', 'AzureDocIntLocal', 'LlamaParse'
pdf_loader = "PyMuPDF"

# Define the PDF to query
# Options: 'Attention Is All You Need.pdf', 'CODIMUR 50.pdf', 'Orden ayudas publicas.pdf', 'PPT Sostenibilidad.pdf', 'UOC Ciencia de Datos.pdf'
k, source = 5, "Attention Is All You Need.pdf"

# Define the RAG Chain
rag_chain = MyRAGChain(
    source=source, 
    pdf_loader=pdf_loader, 
    embedding_model=embedding_model, 
    db_provider=db_provider, 
    llm_model=llm_model, 
    k=k, 
    temperature=temperature
).get_rag_chain()

# Invoke the RAG Chain
quetion = "¿Qué porcentaje de empresarios españoles sitúa la atracción y fidelización del talento como una de sus principales prioridades estratégicas para este año?"
output = rag_chain.invoke(quetion)

# Print the RAG Chain output
print(output["answer"])
print(output["retrievals"])

## Flujo de Evaluación con DeepEval y RAGAs

En esta sección, definimos el flujo de evaluación, que itera sobre todos nuestros pipelines RAG aplicándolos a las consultas del dataset de evaluación. El objetivo es calcular las métricas RAGAs, que permiten medir el rendimiento de los distintos flujos de RAG en función de las respuestas generadas. 

In [ ]:
## ----------------------------------------------------------------------------
## FLUJO DE EVALUACIÓN CON DEEPEVAL Y RAGAS:
# Construcción sintetica de los datasets de evaluación con DeepEval.
## ----------------------------------------------------------------------------
# Nota: El rendimiento no ha sido el esperado, por lo que se ha decidido
# generar los datasets de evaluación de manera manual.
## ----------------------------------------------------------------------------

# PDF Options: 'Attention Is All You Need.pdf', 'CODIMUR 50.pdf', 'Orden ayudas publicas.pdf', 'PPT Sostenibilidad.pdf', 'UOC Ciencia de Datos.pdf'
source = "Attention Is All You Need.pdf"

# PDF Loader Options: 'PyMuPDF', 'UnstructuredCloud', 'AzureDocIntLocal', 'LlamaParse'
pdf_loader = "PyMuPDF"

# Database Provider Options: 'chromadb', 'mongodb'
db_provider = "mongodb"

# Embedding Model Options: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3'
embedding_model = "text-embedding-3-large"

# Select your context to build your dataset from
VectorStore = MyVectorStore(provider=db_provider, Embedder=MyEmbedder(embedding_model))
vector_store = VectorStore.get_vector_store()
documents = list(vector_store._collection.find({"source": source, "parser": pdf_loader}, {"text": 1}))

contexts = [x['text'] for x in documents]
contexts = [contexts[i:i + 5] for i in range(0, len(contexts), 5)]

# Generate Goldens a.k.a. Question & Expected Outputs
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer(model="gpt-4o", max_concurrent=1)
synthesizer.generate_goldens_from_contexts(contexts=contexts, include_expected_output=True, max_goldens_per_context=2)
synthesizer.save_as(file_type='json', directory="../DeepEval")

In [38]:
## ----------------------------------------------------------------------------
## FLUJO DE EVALUACIÓN CON DEEPEVAL Y RAGAS:
# Interfaz de evaluación para Gemini Flash 1.5
# DeepEval requiere de una interfaz distinta a la proporcionada por la 
# librería LangChain.
## ----------------------------------------------------------------------------

import instructor
from pydantic import BaseModel
import google.generativeai as genai

from deepeval.models import DeepEvalBaseLLM

class CustomGeminiFlash(DeepEvalBaseLLM):
    def __init__(self):
        self.model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        client = self.load_model()
        instructor_client = instructor.from_gemini(
            client=client,
            mode=instructor.Mode.GEMINI_JSON,
        )
        resp = instructor_client.messages.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            response_model=schema,
        )
        return resp

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemini 1.5 Flash"

In [39]:
## ----------------------------------------------------------------------------
## FLUJO DE EVALUACIÓN CON DEEPEVAL Y RAGAS:
# Unidad de evaluación de las métricas RAGAs utilizando Gemini Flash 1.5
# Dado un pipeline de RAG y un test case, evalua el pipeline y devuelve 
# los resultados.
## ----------------------------------------------------------------------------

import json

from deepeval import evaluate
from deepeval.test_case import LLMTestCase

from deepeval.metrics import AnswerRelevancyMetric
from deepeval.metrics import FaithfulnessMetric
from deepeval.metrics import ContextualRecallMetric
from deepeval.metrics import ContextualPrecisionMetric

def gemini_evaluate_test_case(RAGChain: MyRAGChain, test_case: dict, replace: bool = False):

    # If replace=False and already evaluated do not evaluate again
    evaluation_json = f"../DeepEval/{RAGChain.source.split('.')[0]} - Evaluation.json"
    with open(evaluation_json, 'r') as f:
        existing_evaluation_list = json.load(f)
    
    evaluation_item = [evaluation_item for evaluation_item in existing_evaluation_list 
                       if evaluation_item['input'] == test_case['input']
                       and evaluation_item['llm_model'] == RAGChain.llm_model
                       and evaluation_item['embedding_model'] == RAGChain.embedding_model
                       and evaluation_item['pdf_loader'] == RAGChain.pdf_loader
                       and evaluation_item['db_provider'] == RAGChain.db_provider]
    
    
    if evaluation_item and not replace:
        return evaluation_item[0]
    
    elif evaluation_item and replace:
        existing_evaluation_list = [item for item in existing_evaluation_list 
                                    if not (item['input'] == evaluation_item[0]['input']
                                    and item['llm_model'] == RAGChain.llm_model
                                    and item['embedding_model'] == RAGChain.embedding_model
                                    and item['pdf_loader'] == RAGChain.pdf_loader
                                    and item['db_provider'] == RAGChain.db_provider)]
    
    # Run the RAG Chain to generate the real output
    output = RAGChain.get_rag_chain().invoke(test_case['input'])

    # Build the LLM Test Case
    llm_test_case = LLMTestCase(
        input=test_case['input'],
        actual_output=output['answer'],
        expected_output=test_case['expected_output'],
        retrieval_context=[doc.page_content for doc in output['retrievals']]
    )

    # Evaluate the LLM Test Case
    evaluation_model =  CustomGeminiFlash()
    metric_AnswerRelevancyMetric = AnswerRelevancyMetric(threshold=0.7, model=evaluation_model, include_reason=False)
    metric_FaithfulnessMetric = FaithfulnessMetric(threshold=0.7, model=evaluation_model, include_reason=False)
    metric_ContextualRecallMetric = ContextualRecallMetric(threshold=0.7, model=evaluation_model, include_reason=False)
    metric_ContextualPrecisionMetric = ContextualPrecisionMetric(threshold=0.7, model=evaluation_model, include_reason=False)

    evaluation_results = evaluate(
        [llm_test_case], 
        [metric_AnswerRelevancyMetric, metric_FaithfulnessMetric, metric_ContextualRecallMetric, metric_ContextualPrecisionMetric], 
        verbose_mode=False,
        print_results=False
    )
    
    evaluation_scores = [{"metric":metric.name, "score":metric.score} for metric in evaluation_results.test_results[0].metrics_data]
    average_score = sum(item['score'] for item in evaluation_scores) / len(evaluation_scores)

    # Save Evaluation Results
    evaluation_item = {
        "input": test_case['input'],
        "actual_output": output['answer'],
        "expected_output": test_case['expected_output'],
        "retrieval_context": [doc.page_content for doc in output['retrievals']],
        "context": test_case['context'],
        "source": source,
        "pdf_loader": pdf_loader,
        "embedding_model": embedding_model,
        "db_provider": db_provider,
        "llm_model": llm_model,
        "Contextual Precision": round(next(item['score'] for item in evaluation_scores if item['metric'] == 'Contextual Precision'), 3),
        "Contextual Recall": round(next(item['score'] for item in evaluation_scores if item['metric'] == 'Contextual Recall'), 3),
        "Answer Relevancy": round(next(item['score'] for item in evaluation_scores if item['metric'] == 'Answer Relevancy'), 3),
        "Faithfulness": round(next(item['score'] for item in evaluation_scores if item['metric'] == 'Faithfulness'), 3),
        "RAGAs Average": round(average_score, 3)
    }

    existing_evaluation_list.extend([evaluation_item])
                    
    with open(evaluation_json, 'w') as f:
        json.dump(existing_evaluation_list, f)
    
    return evaluation_item

In [ ]:
## ----------------------------------------------------------------------------
## FLUJO DE EVALUACIÓN CON DEEPEVAL Y RAGAS:
# Evaluación de los pipelines de RAG con Gemini Flash 1.5
## ----------------------------------------------------------------------------

import os
import json

# For all the pdfs
# Available PDFs: 'Attention Is All You Need.pdf', 'CODIMUR 50.pdf', 'Orden Ayudas Publicas.pdf', 'PPT Sostenibilidad.pdf', 'UOC Ciencia de Datos.pdf'
pdfs_list = [r"../PDFs/PPT Sostenibilidad.pdf", r"../PDFs/UOC Ciencia de Datos.pdf"]
for path in pdfs_list:

    source = os.path.basename(path)
    file_name = source.split(".")[0]

    dataset_json = f"../DeepEval/{file_name} - Dataset.json"
    evaluation_json = f"../DeepEval/{file_name} - Evaluation.json"

    if not os.path.exists(evaluation_json):
        with open(evaluation_json, 'w') as f:
            json.dump([], f)
      
    # For all the pdf loaders
    # Available loaders: 'PyMuPDF', 'UnstructuredCloud', 'AzureDocIntLocal', 'LlamaParse'
    pdf_loader_list = ["PyMuPDF", "UnstructuredCloud", "AzureDocIntLocal", "LlamaParse"]
    for pdf_loader in pdf_loader_list:

        # For all the embedding models
        # Available models: 'nomic-embed-text', 'snowflake-arctic-embed-335m', 'text-embedding-3-large', 'voyage-3'
        embedding_models_list = ["nomic-embed-text", "snowflake-arctic-embed-335m", "text-embedding-3-large", "voyage-3"]
        for embedding_model in embedding_models_list:

            # For all the database providers
            # Available providers: 'chromadb', 'mongodb'
            db_providers_list = ["chromadb", "mongodb"]
            for db_provider in db_providers_list:

                # For all the language models
                # Available models: 'gpt-4o', 'claude-3-5-sonnet', 'gemini-1.5-pro', 'gemma2-9b', 'llama3-1-8b', 'phi3-5'
                llm_model_list = ["gpt-4o", "claude-3-5-sonnet", "gemini-1.5-pro", "gemma2-9b", "llama3-1-8b", "phi3-5"]
                for llm_model in llm_model_list:

                    print(f"Evaluating {llm_model} on {os.path.basename(path)} with {pdf_loader}, {embedding_model} and {db_provider} ...")

                    # --------------------------
                    # Instantiate the RAG Chain
                    # --------------------------
                    RAGChain = MyRAGChain(
                        source=source, 
                        pdf_loader=pdf_loader, 
                        embedding_model=embedding_model, 
                        db_provider=db_provider, 
                        llm_model=llm_model, 
                        k=5, 
                        temperature=0.5
                    )
                    # --------------------------
                    
                    # Load the evaluation dataset
                    with open(dataset_json, 'r') as f:
                        dataset_list = json.load(f)
                    
                    # For all the test cases in the dataset
                    for ii, test_case in enumerate(dataset_list):
                        # --------------------------
                        # Evaluate the RAG Chain
                        # --------------------------
                        try:
                            evaluation_item = gemini_evaluate_test_case(RAGChain=RAGChain, test_case=test_case, replace=False)
                            print(f"{ii+1} de {len(dataset_list)}:: {llm_model} on {os.path.basename(path)} with {pdf_loader}, {embedding_model} and {db_provider}")
                            print(f"Question: {evaluation_item['input']}")
                            print(f"Answer: {evaluation_item['actual_output']}")
                        except Exception as e:
                            print(f"Error: {e}")
                            continue
                        # --------------------------

                    print(f"Evaluating {llm_model} on {os.path.basename(path)} with {pdf_loader}, {embedding_model} and {db_provider} ... DONE")

Evaluating gpt-4o on PPT Sostenibilidad.pdf with PyMuPDF, nomic-embed-text and chromadb ...
1 de 12:: gpt-4o on PPT Sostenibilidad.pdf with PyMuPDF, nomic-embed-text and chromadb
Question: ¿Qué porcentaje de empresarios españoles sitúa la atracción y fidelización del talento como una de sus principales prioridades estratégicas para este año?
Answer: El 60% de los empresarios españoles sitúa la atracción y fidelización del talento como una de sus principales prioridades estratégicas para este año.
2 de 12:: gpt-4o on PPT Sostenibilidad.pdf with PyMuPDF, nomic-embed-text and chromadb
Question: ¿Qué directiva va a facilitar la reflexión de las compañías del sector sobre cuestiones de sostenibilidad?
Answer: La directiva que va a facilitar la reflexión de las compañías del sector sobre cuestiones de sostenibilidad es la directiva de información sobre sostenibilidad (CSRD).
3 de 12:: gpt-4o on PPT Sostenibilidad.pdf with PyMuPDF, nomic-embed-text and chromadb
Question: ¿Qué porcentaje de la

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Gemini 1.5 Flash, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using Gemini 1.5 Flash, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Gemini 1.5 Flash, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Gemini 1.5 Flash, strict=False, 
async_mode=True)...

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:57, 57.80s/test case]


Results saved in ../DeepEval/TestResults as 20241217_212450

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

2 de 12:: phi3-5 on PPT Sostenibilidad.pdf with PyMuPDF, nomic-embed-text and mongodb
Question: ¿Qué directiva va a facilitar la reflexión de las compañías del sector sobre cuestiones de sostenibilidad?
Answer: La reciente Directiva de Información Sobre Sostenibilidad (CSRD) va a facilitar la reflexión de las compañías del sector sobre cuestiones de sostenibilidad.


## Análisis de los Resultados - Pendiente ...

In [ ]:
import os
import json
import numpy as np

In [ ]:
pdf_path = r"../PDFs/Attention Is All You Need.pdf"
source = os.path.basename(pdf_path)
file_name = source.split(".")[0]

In [ ]:
evaluation_json = f"../DeepEval/{file_name} - Evaluation.json"
with open(evaluation_json, 'r') as f:
    evaluation_list = json.load(f)

In [ ]:
def overall_average(evaluation_list: list, key: str, value: str):
    filtered_list = [x for x in evaluation_list if x[key] == value]

    contextual_precision_values = [x['Contextual Precision'] for x in filtered_list if not np.isnan(x['Contextual Precision'])]
    contextual_recall_values = [x['Contextual Recall'] for x in filtered_list if not np.isnan(x['Contextual Recall'])]
    answer_relevancy_values = [x['Answer Relevancy'] for x in filtered_list if not np.isnan(x['Answer Relevancy'])]
    faithfulness_values = [x['Faithfulness'] for x in filtered_list if not np.isnan(x['Faithfulness'])]
    ragas_average_values = [x['RAGAs Average'] for x in filtered_list if not np.isnan(x['RAGAs Average'])]

    contextual_precision = round(sum(contextual_precision_values) / len(contextual_precision_values), 3)
    contextual_recall = round(sum(contextual_recall_values) / len(contextual_recall_values), 3)
    answer_relevancy = round(sum(answer_relevancy_values) / len(answer_relevancy_values), 3)
    faithfulness = round(sum(faithfulness_values) / len(faithfulness_values), 3)
    ragas_average = round(sum(ragas_average_values) / len(ragas_average_values), 3)

    return [contextual_precision, contextual_recall, answer_relevancy, faithfulness, ragas_average]

In [ ]:
# -----------------------------------
# Loaders - Overall Average
# -----------------------------------

PyMuPDF_Results = overall_average(evaluation_list, 'pdf_loader', "PyMuPDF")
print(PyMuPDF_Results)

AzureDocIntLocal_Results = overall_average(evaluation_list, 'pdf_loader', "AzureDocIntLocal")
print(AzureDocIntLocal_Results)

LlamaParse_Results = overall_average(evaluation_list, 'pdf_loader', "LlamaParse")
print(LlamaParse_Results)

In [ ]:
# -----------------------------------
# Embedders - Overall Average
# -----------------------------------

NomicEmbedText_Results = overall_average(evaluation_list, 'embedding_model', "nomic-embed-text")
print(NomicEmbedText_Results)

TextEmbedding3Large_Results = overall_average(evaluation_list, 'embedding_model', "text-embedding-3-large")
print(TextEmbedding3Large_Results)

In [ ]:
# -----------------------------------
# Databases - Overall Average
# -----------------------------------

Chromadb_Results = overall_average(evaluation_list, 'db_provider', "chromadb")
print(Chromadb_Results)

Mongodb_Results = overall_average(evaluation_list, 'db_provider', "mongodb")
print(Mongodb_Results)

In [ ]:
# -----------------------------------
# Language Models - Overall Average
# -----------------------------------

GPT4o_Results = overall_average(evaluation_list, 'llm_model', "gpt-4o")
print(GPT4o_Results)

Claude35Sonnet_Results = overall_average(evaluation_list, 'llm_model', "claude-3-5-sonnet")
print(Claude35Sonnet_Results)

Gemini15Pro_Results = overall_average(evaluation_list, 'llm_model', "gemini-1.5-pro")
print(Gemini15Pro_Results)

Gemma29b_Results = overall_average(evaluation_list, 'llm_model', "gemma2-9b")
print(Gemma29b_Results)

Llama318b_Results = overall_average(evaluation_list, 'llm_model', "llama3-1-8b")
print(Llama318b_Results)

Phi35_Results = overall_average(evaluation_list, 'llm_model', "phi3-5")
print(Phi35_Results)

In [ ]:
def env_average(evaluation_list: list, env: str):
    pdf_loaders_local, pdf_loaders_cloud = ["PyMuPDF", "AzureDocIntLocal"], ["LlamaParse"]
    embedding_models_local, embedding_models_cloud = ["nomic-embed-text"], ["text-embedding-3-large"]
    db_providers_local, db_providers_cloud = ["chromadb"], ["mongodb"]
    llm_models_local, llm_models_cloud = ["gemma2-9b", "llama3-1-8b", "phi3-5"], ["gpt-4o", "claude-3-5-sonnet", "gemini-1.5-pro"]
    
    if env == "local":
        filtered_list = [x for x in evaluation_list
                                if (x['pdf_loader'] in pdf_loaders_local)
                                and (x['embedding_model'] in embedding_models_local)
                                and (x['db_provider'] in db_providers_local)
                                and (x['llm_model'] in llm_models_local)]
    
    elif env == "cloud":
        filtered_list = [x for x in evaluation_list
                                if (x['pdf_loader'] in pdf_loaders_cloud)
                                and (x['embedding_model'] in embedding_models_cloud)
                                and (x['db_provider'] in db_providers_cloud)
                                and (x['llm_model'] in llm_models_cloud)]


    contextual_precision_values = [x['Contextual Precision'] for x in filtered_list if not np.isnan(x['Contextual Precision'])]
    contextual_recall_values = [x['Contextual Recall'] for x in filtered_list if not np.isnan(x['Contextual Recall'])]
    answer_relevancy_values = [x['Answer Relevancy'] for x in filtered_list if not np.isnan(x['Answer Relevancy'])]
    faithfulness_values = [x['Faithfulness'] for x in filtered_list if not np.isnan(x['Faithfulness'])]
    ragas_average_values = [x['RAGAs Average'] for x in filtered_list if not np.isnan(x['RAGAs Average'])]

    contextual_precision = round(sum(contextual_precision_values) / len(contextual_precision_values), 3)
    contextual_recall = round(sum(contextual_recall_values) / len(contextual_recall_values), 3)
    answer_relevancy = round(sum(answer_relevancy_values) / len(answer_relevancy_values), 3)
    faithfulness = round(sum(faithfulness_values) / len(faithfulness_values), 3)
    ragas_average = round(sum(ragas_average_values) / len(ragas_average_values), 3)

    return [contextual_precision, contextual_recall, answer_relevancy, faithfulness, ragas_average]

In [ ]:
# -----------------------------------
# Local vs Cloud Average
# -----------------------------------

Local_Results = env_average(evaluation_list, "local")   
print(Local_Results)

Cloud_Results = env_average(evaluation_list, "cloud")
print(Cloud_Results)

In [ ]:
def average_within_environment(evaluation_list: list, key: str, value: str):
    pdf_loaders_local, pdf_loaders_cloud = ["PyMuPDF", "AzureDocIntLocal"], ["LlamaParse"]
    embedding_models_local, embedding_models_cloud = ["nomic-embed-text"], ["text-embedding-3-large"]
    db_providers_local, db_providers_cloud = ["chromadb"], ["mongodb"]
    llm_models_local, llm_models_cloud = ["gemma2-9b", "llama3-1-8b", "phi3-5"], ["gpt-4o", "claude-3-5-sonnet", "gemini-1.5-pro"]

    local_list = pdf_loaders_local + embedding_models_local + db_providers_local + llm_models_local
    cloud_list = pdf_loaders_cloud + embedding_models_cloud + db_providers_cloud + llm_models_cloud

    if value in local_list:
        env_filtered_list = [x for x in evaluation_list
                                if (x['pdf_loader'] in pdf_loaders_local)
                                and (x['embedding_model'] in embedding_models_local)
                                and (x['db_provider'] in db_providers_local)
                                and (x['llm_model'] in llm_models_local)]
    
    elif value in cloud_list:
        env_filtered_list = [x for x in evaluation_list
                                if (x['pdf_loader'] in pdf_loaders_cloud)
                                and (x['embedding_model'] in embedding_models_cloud)
                                and (x['db_provider'] in db_providers_cloud)
                                and (x['llm_model'] in llm_models_cloud)]

    filtered_list = [x for x in env_filtered_list if x[key] == value]

    contextual_precision_values = [x['Contextual Precision'] for x in filtered_list if not np.isnan(x['Contextual Precision'])]
    contextual_recall_values = [x['Contextual Recall'] for x in filtered_list if not np.isnan(x['Contextual Recall'])]
    answer_relevancy_values = [x['Answer Relevancy'] for x in filtered_list if not np.isnan(x['Answer Relevancy'])]
    faithfulness_values = [x['Faithfulness'] for x in filtered_list if not np.isnan(x['Faithfulness'])]
    ragas_average_values = [x['RAGAs Average'] for x in filtered_list if not np.isnan(x['RAGAs Average'])]

    contextual_precision = round(sum(contextual_precision_values) / len(contextual_precision_values), 3)
    contextual_recall = round(sum(contextual_recall_values) / len(contextual_recall_values), 3)
    answer_relevancy = round(sum(answer_relevancy_values) / len(answer_relevancy_values), 3)
    faithfulness = round(sum(faithfulness_values) / len(faithfulness_values), 3)
    ragas_average = round(sum(ragas_average_values) / len(ragas_average_values), 3)

    return [contextual_precision, contextual_recall, answer_relevancy, faithfulness, ragas_average]

In [ ]:
# -----------------------------------
# Loaders - Average Within Environment
# -----------------------------------

PyMuPDF_EnvResults = average_within_environment(evaluation_list, 'pdf_loader', "PyMuPDF")
print(PyMuPDF_EnvResults)

AzureDocIntLocal_EnvResults = average_within_environment(evaluation_list, 'pdf_loader', "AzureDocIntLocal")
print(AzureDocIntLocal_EnvResults)

LlamaParse_EnvResults = average_within_environment(evaluation_list, 'pdf_loader', "LlamaParse")
print(LlamaParse_EnvResults)

In [ ]:
# -----------------------------------
# Embedders - Average Within Environment
# -----------------------------------

NomicEmbedText_EnvResults = average_within_environment(evaluation_list, 'embedding_model', "nomic-embed-text")
print(NomicEmbedText_EnvResults)

TextEmbedding3Large_EnvResults = average_within_environment(evaluation_list, 'embedding_model', "text-embedding-3-large")
print(TextEmbedding3Large_EnvResults)

In [ ]:
# -----------------------------------
# Databases - Average Within Environment
# -----------------------------------

Chromadb_EnvResults = average_within_environment(evaluation_list, 'db_provider', "chromadb")
print(Chromadb_EnvResults)

Mongodb_EnvResults = average_within_environment(evaluation_list, 'db_provider', "mongodb")
print(Mongodb_EnvResults)

In [ ]:
# -----------------------------------
# Language Models - Average Within Environment
# -----------------------------------

GPT4o_EnvResults = average_within_environment(evaluation_list, 'llm_model', "gpt-4o")
print(GPT4o_EnvResults)

Claude35Sonnet_EnvResults = average_within_environment(evaluation_list, 'llm_model', "claude-3-5-sonnet")
print(Claude35Sonnet_EnvResults)

Gemini15Pro_EnvResults = average_within_environment(evaluation_list, 'llm_model', "gemini-1.5-pro")
print(Gemini15Pro_EnvResults)

Gemma29b_EnvResults = average_within_environment(evaluation_list, 'llm_model', "gemma2-9b")
print(Gemma29b_EnvResults)

Llama318b_EnvResults = average_within_environment(evaluation_list, 'llm_model', "llama3-1-8b")
print(Llama318b_EnvResults)

Phi35_EnvResults = average_within_environment(evaluation_list, 'llm_model', "phi3-5")
print(Phi35_EnvResults)